# GPT 고객 서비스 챗봇 만들기

In [1]:
# 터미널에서 실행시
# python -m venv venv
# source venv/bin/activate  # 윈도우는 venv\Scripts\activate
# pip install -r requirements.txt
# python app.py

In [1]:
from dotenv import load_dotenv
import os

# .env 파일 불러오기
load_dotenv("C:/env/.env")

# 환경 변수 가져오기
API_KEY = os.getenv("OPENAI_API_KEY")

from openai import OpenAI
client = OpenAI(api_key=API_KEY)

from flask import Flask, jsonify, request, render_template
app = Flask(__name__)

In [2]:
@app.route('/')
def home():
    return render_template('index.html')

@app.route('/get_response', methods=['POST'])
def get_response():
    user_message = request.json.get("message")

    #  OpenAI 모델에 요청
    response = client.chat.completions.create(
        model = "gpt-4o-mini", 
        max_tokens= 300,
        messages = [ {"role":"user", "content":user_message} ],
        n = 1,
        stop = None,
        temperature = 0.7,
    )    

    bot_response = response.choices[0].message.content
    return jsonify({"response": bot_response})
   
           
if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


## 질문한 내용을 Context로 계속 유지하면서 질문하기

In [3]:
# 소스 코드를 이어서 실행할 경우 오류나므로 "Restart Kernel"을 수행하고 실행한다

@app.route('/')
def home():
    return render_template('index.html')

# 대화 히스토리를 저장할 변수
conversation_history = []

@app.route('/get_response', methods=['POST'])
def get_response():
    user_message = request.json.get("message")
    conversation_history.append({"role":"user","content":user_message})
    print(conversation_history)

    # OpenAI 모델에 요청
    response = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages = conversation_history,
        max_tokens = 300,
        n = 1,
        stop = None,
        temperature = 0.2,
    )

    bot_response = response.choices[0].message.content
    return jsonify({"response": bot_response})
        
if __name__ == '__main__':
    app.run(debug=False)    

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [29/Sep/2025 19:03:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2025 19:03:46] "GET /favicon.ico HTTP/1.1" 404 -


[{'role': 'user', 'content': '나는 홍길동이야'}]


127.0.0.1 - - [29/Sep/2025 19:03:58] "POST /get_response HTTP/1.1" 200 -


[{'role': 'user', 'content': '나는 홍길동이야'}, {'role': 'user', 'content': '내이름이 뭐지?'}]


127.0.0.1 - - [29/Sep/2025 19:04:07] "POST /get_response HTTP/1.1" 200 -


[{'role': 'user', 'content': '나는 홍길동이야'}, {'role': 'user', 'content': '내이름이 뭐지?'}, {'role': 'user', 'content': '하늘이 파란 이유는?'}]


127.0.0.1 - - [29/Sep/2025 19:04:31] "POST /get_response HTTP/1.1" 200 -


[{'role': 'user', 'content': '나는 홍길동이야'}, {'role': 'user', 'content': '내이름이 뭐지?'}, {'role': 'user', 'content': '하늘이 파란 이유는?'}, {'role': 'user', 'content': '노란 이유는?'}]


127.0.0.1 - - [29/Sep/2025 19:04:44] "POST /get_response HTTP/1.1" 200 -


### 질문과 답변한 내용을 Context로 계속 유지하면서 질문하기 

In [ ]:
# 소스 코드를 이어서 실행할 경우 오류나므로 "Restart Kernel"을 수행하고 실행한다

@app.route('/')
def home():
    return render_template('index.html')

# 대화 히스토리를 저장할 변수
conversation_history = []

@app.route('/get_response', methods=['POST'])
def get_response():
    user_message = request.json.get("message")
    
    context = f"대화 내용을 참조해서 답변하세요 : {conversation_history}"

    # OpenAI 모델에 요청
    response = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages = [  {"role": "system", "content": context},
                      {"role": "user", "content": user_message} ],           

        max_tokens = 300,
        n = 1,
        stop = None,
        temperature = 0.7,
    )

    bot_response = response.choices[0].message.content

    # 대화 내용을 history에 저장
    conversation_history.append({"질문":user_message,"답변":bot_response})
    print('history:\n',conversation_history,'\n')
    print('-'*30)
    
    return jsonify({"response": bot_response})
        
if __name__ == '__main__':
    app.run(debug=False)    

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [29/Sep/2025 19:19:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2025 19:20:14] "POST /get_response HTTP/1.1" 200 -


history:
 [{'질문': '하늘이 파란이유는?', '답변': '하늘이 파란 이유는 대기 중의 산란 현상 때문입니다. 태양빛은 여러 색의 빛으로 구성되어 있는데, 이 중 파란색 빛은 다른 색에 비해 파장이 짧아 대기 중의 분자와 더 많이 산란됩니다. 이 산란 현상으로 인해 하늘을 바라볼 때 파란색이 주로 보이게 됩니다. 특히, 태양이 높이 떠 있을 때 더욱 뚜렷하게 나타나며, 해가 지거나 해가 뜰 때는 다른 색의 빛이 더 많이 산란되어 붉은색이나 주황색으로 보이기도 합니다.'}] 

------------------------------


127.0.0.1 - - [29/Sep/2025 19:20:29] "POST /get_response HTTP/1.1" 200 -


history:
 [{'질문': '하늘이 파란이유는?', '답변': '하늘이 파란 이유는 대기 중의 산란 현상 때문입니다. 태양빛은 여러 색의 빛으로 구성되어 있는데, 이 중 파란색 빛은 다른 색에 비해 파장이 짧아 대기 중의 분자와 더 많이 산란됩니다. 이 산란 현상으로 인해 하늘을 바라볼 때 파란색이 주로 보이게 됩니다. 특히, 태양이 높이 떠 있을 때 더욱 뚜렷하게 나타나며, 해가 지거나 해가 뜰 때는 다른 색의 빛이 더 많이 산란되어 붉은색이나 주황색으로 보이기도 합니다.'}, {'질문': '노란 이유는?', '답변': '하늘이 노란색으로 보이는 이유는 주로 태양의 위치와 대기 중의 산란 현상 때문입니다. 해가 지거나 뜰 때, 태양빛이 지평선을 통해 지나갈 때 대기 중의 먼지와 수분 같은 입자에 의해 빛이 더 많이 산란됩니다. 이때 파장이 긴 붉은색과 주황색 빛이 더 많이 산란되어 하늘이 노란색으로 보이게 됩니다. 또한, 대기 중에 있는 오염물질이나 미세먼지가 노란색을 더욱 강조할 수도 있습니다.'}] 

------------------------------


...

history:
 [{'질문': '하늘이 파란이유는?', '답변': '하늘이 파란 이유는 대기 중의 산란 현상 때문입니다. 태양빛은 여러 색의 빛으로 구성되어 있는데, 이 중 파란색 빛은 다른 색에 비해 파장이 짧아 대기 중의 분자와 더 많이 산란됩니다. 이 산란 현상으로 인해 하늘을 바라볼 때 파란색이 주로 보이게 됩니다. 특히, 태양이 높이 떠 있을 때 더욱 뚜렷하게 나타나며, 해가 지거나 해가 뜰 때는 다른 색의 빛이 더 많이 산란되어 붉은색이나 주황색으로 보이기도 합니다.'}, {'질문': '노란 이유는?', '답변': '하늘이 노란색으로 보이는 이유는 주로 태양의 위치와 대기 중의 산란 현상 때문입니다. 해가 지거나 뜰 때, 태양빛이 지평선을 통해 지나갈 때 대기 중의 먼지와 수분 같은 입자에 의해 빛이 더 많이 산란됩니다. 이때 파장이 긴 붉은색과 주황색 빛이 더 많이 산란되어 하늘이 노란색으로 보이게 됩니다. 또한, 대기 중에 있는 오염물질이나 미세먼지가 노란색을 더욱 강조할 수도 있습니다.'}, {'질문': '나는 홍길동이야', '답변': '안녕하세요, 홍길동님! 어떻게 도와드릴까요?'}] 

------------------------------


127.0.0.1 - - [29/Sep/2025 19:21:22] "POST /get_response HTTP/1.1" 200 -


history:
 [{'질문': '하늘이 파란이유는?', '답변': '하늘이 파란 이유는 대기 중의 산란 현상 때문입니다. 태양빛은 여러 색의 빛으로 구성되어 있는데, 이 중 파란색 빛은 다른 색에 비해 파장이 짧아 대기 중의 분자와 더 많이 산란됩니다. 이 산란 현상으로 인해 하늘을 바라볼 때 파란색이 주로 보이게 됩니다. 특히, 태양이 높이 떠 있을 때 더욱 뚜렷하게 나타나며, 해가 지거나 해가 뜰 때는 다른 색의 빛이 더 많이 산란되어 붉은색이나 주황색으로 보이기도 합니다.'}, {'질문': '노란 이유는?', '답변': '하늘이 노란색으로 보이는 이유는 주로 태양의 위치와 대기 중의 산란 현상 때문입니다. 해가 지거나 뜰 때, 태양빛이 지평선을 통해 지나갈 때 대기 중의 먼지와 수분 같은 입자에 의해 빛이 더 많이 산란됩니다. 이때 파장이 긴 붉은색과 주황색 빛이 더 많이 산란되어 하늘이 노란색으로 보이게 됩니다. 또한, 대기 중에 있는 오염물질이나 미세먼지가 노란색을 더욱 강조할 수도 있습니다.'}, {'질문': '나는 홍길동이야', '답변': '안녕하세요, 홍길동님! 어떻게 도와드릴까요?'}, {'질문': '내이름이 뭐지', '답변': '당신의 이름은 홍길동입니다! 어떻게 도와드릴까요?'}] 

------------------------------


127.0.0.1 - - [29/Sep/2025 19:22:00] "POST /get_response HTTP/1.1" 200 -


history:
 [{'질문': '하늘이 파란이유는?', '답변': '하늘이 파란 이유는 대기 중의 산란 현상 때문입니다. 태양빛은 여러 색의 빛으로 구성되어 있는데, 이 중 파란색 빛은 다른 색에 비해 파장이 짧아 대기 중의 분자와 더 많이 산란됩니다. 이 산란 현상으로 인해 하늘을 바라볼 때 파란색이 주로 보이게 됩니다. 특히, 태양이 높이 떠 있을 때 더욱 뚜렷하게 나타나며, 해가 지거나 해가 뜰 때는 다른 색의 빛이 더 많이 산란되어 붉은색이나 주황색으로 보이기도 합니다.'}, {'질문': '노란 이유는?', '답변': '하늘이 노란색으로 보이는 이유는 주로 태양의 위치와 대기 중의 산란 현상 때문입니다. 해가 지거나 뜰 때, 태양빛이 지평선을 통해 지나갈 때 대기 중의 먼지와 수분 같은 입자에 의해 빛이 더 많이 산란됩니다. 이때 파장이 긴 붉은색과 주황색 빛이 더 많이 산란되어 하늘이 노란색으로 보이게 됩니다. 또한, 대기 중에 있는 오염물질이나 미세먼지가 노란색을 더욱 강조할 수도 있습니다.'}, {'질문': '나는 홍길동이야', '답변': '안녕하세요, 홍길동님! 어떻게 도와드릴까요?'}, {'질문': '내이름이 뭐지', '답변': '당신의 이름은 홍길동입니다! 어떻게 도와드릴까요?'}, {'질문': '빨간이유는?', '답변': '하늘이 빨간색으로 보이는 이유는 주로 태양의 위치와 대기 중의 산란 현상 때문입니다. 해가 지거나 뜰 때, 태양빛이 지평선을 통해 지나가면서 대기 중의 먼지, 수분, 그리고 입자들에 의해 빛이 더 많이 산란됩니다. 이때 파장이 긴 붉은색 빛이 더 많이 산란되어 하늘이 빨간색으로 보이게 됩니다. 또한, 대기 중에 있는 오염물질이나 미세먼지가 빨간색을 더욱 강조할 수 있습니다.'}] 

------------------------------
